# Transfer Learning CNN Test

Creation date: March 2022

Edit date: March 31, 2022

## Objectives

* Synthetic data
* Improve modularity of code
* Create df to log experiments
* Test other models to resnet50

In [1]:
import os
from os import listdir
import time

# Torch libs
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from torch.utils.data.sampler import WeightedRandomSampler

import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torchvision.io import read_image
import torch.optim as optim


from sklearn.model_selection import train_test_split

# Data libs
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw

### From Dan:

from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler, random_split
import torch.nn as nn

from PIL import Image
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import xml.etree.ElementTree as ET

In [2]:
!pwd

/home/ec2-user/git/doggo_vision/experiments/felix


In [3]:
os.chdir('..')
os.chdir('..')
os.chdir('dogs/Images')

In [4]:
######################## REPLACE WITH YOUR NAME TO LOCATE FILES IN YOUR REPO ########################
YOUR_NAME="felix"

# Why is model worse than dan's
# pay special attention to the dataset. data portion.
# Training mechanism compared to what I'm doing.

In [5]:
# # Create labels dataframe and save to csv
# labels = ['Chihuahua']
# image_path = []
# label_num = []

# label_count = 0
# for root, dirs, files in os.walk("."):
#     for name in files:
#         labels.append(root.split("-",1)[1])
#         image_path.append(os.path.join(root, name))
        
#         if labels[-1] != labels[-2]:
#             label_count += 1
        
#         label_num.append(label_count)
        

# df = pd.DataFrame({'labels':labels[1:],'image_path':image_path, 'label_num':label_num}) 

# # Create label dict for later use
# breeds = pd.Series(df.labels.values,index=df.label_num).to_dict()

# # saving the dataframe 
# df.to_csv("../../experiments/"+YOUR_NAME+"/annotations.csv") 
# display(df.head())
# display(df.tail())

In [6]:
# Test labels
doggo_labels = pd.read_csv("../../experiments/"+YOUR_NAME+"/annotations.csv")
index = 400

print("Index: ", doggo_labels.iloc[index, 0], "\nLabel: ", doggo_labels.iloc[index, 1], "\nImage directory: ", doggo_labels.iloc[index, 2], "\nLabel number: ", doggo_labels.iloc[index, 3])
print("\nTotal number of samples: ", doggo_labels.shape[0], "\nDog breeds/ unique labels: ", len(pd.unique(doggo_labels['labels'])))
print("\nNumber images per breed \nMax: ", max(doggo_labels['labels'].value_counts()), "\nMin:", min(doggo_labels['labels'].value_counts()))

Index:  400 
Label:  Maltese_dog 
Image directory:  ./n02085936-Maltese_dog/n02085936_426.jpg 
Label number:  2

Total number of samples:  20580 
Dog breeds/ unique labels:  120

Number images per breed 
Max:  252 
Min: 148


In [7]:
# Create train, validate and test with proportional classes

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

label_num = np.array(doggo_labels.iloc[:, 3])

# labels
dataY = label_num

# features
dataX_dummy = range(len(label_num)) 

# train is now 75% of the entire data set
index_train, index_test, y_train, y_test = train_test_split(dataX_dummy, dataY, test_size=1 - train_ratio, stratify=dataY)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
index_val, index_test, y_val, y_test = train_test_split(index_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=y_test) 


In [8]:
# # Check indices work
# img_annotations = pd.read_csv("../../experiments/"+YOUR_NAME+"/annotations.csv").iloc[index_train]
# display(img_annotations.head())
# print(img_annotations.iloc[0,2])
# print(img_annotations.iloc[1,2])

In [9]:
torch.manual_seed(0)

imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
n_pixels = 300
n_degrees = 15

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=n_pixels+15, scale=(0.95, 1.0)),
    transforms.RandomRotation(degrees=n_degrees),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.CenterCrop(size=n_pixels),  
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats) 
]) #these values come from ImageNet Dataset mean and std (resnet50 trained on ImageNet)    
# transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),


# Other transforms?

transform_val = transforms.Compose([
    transforms.Resize(size=n_pixels),
    transforms.CenterCrop(size=n_pixels),
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats) 
#     transforms.HorizontalFlip(),
#     transforms.RandomRotation(degrees = (0,180)),
#     transforms.RandomVerticalFlip()
]) #these values come from ImageNet Dataset mean and std (resnet50 trained on ImageNet)

# Other transforms?


In [10]:
# Experiment 1 - just resize
class DoggoDataset(Dataset):
    def __init__(self, indices, transform=None):
        self.img_annotations = pd.read_csv("../../experiments/"+YOUR_NAME+"/annotations.csv").iloc[indices]
        self.transform = transform

    def __len__(self):
        return len(self.img_annotations)

    def __getitem__(self, idx):
        img_path = self.img_annotations.iloc[idx, 2]
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = self.img_annotations.iloc[idx, 3]

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
train_set = DoggoDataset(index_train, transform_train)
val_set = DoggoDataset(index_val, transform_val)
test_set = DoggoDataset(index_test)

num_workers=4
batch_size = 128
train_dataloader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)

## Custom functions

Using functions created by Chinmay and Nora

In [12]:
def get_accuracy(pred, true):
    # Calculating accuracy by comparing predictions with true labels
    acc = [1 if pred[i] == true[i] else 0 for i in range(len(pred))]
    # Compute accuracy
    acc = np.sum(acc) / len(pred)

    return acc 

In [13]:
def train_one_epoch(train_dataloader):
    
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    model.train()
    
    for images, labels in train_dataloader:
        
        # ResNet is trained on images with only 3 channels
        if(images.shape[1] == 3):
            
            # Load images and labels to device - in our case GPU!
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.to(torch.long)
        
            # Reseting Gradients
            optimizer.zero_grad()

            # Forward
            with torch.set_grad_enabled(True):
                preds = model(images)
                _, max_pred = torch.max(preds, 1)

            # Calculating Loss
            _loss = criterion(preds, labels)
            loss = _loss.item()
            epoch_loss.append(loss)

            # Calculating Accuracy
            acc = get_accuracy(max_pred, labels)
            epoch_acc.append(acc)

            # Backward
            _loss.backward()
            optimizer.step()
            
    lr_scheduler.step()
    # Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    # Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    # Log the results
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [14]:
def val_one_epoch(val_dataloader, best_val_acc):
    
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    model.eval()
    
    for images, labels in val_dataloader:
        
        # ResNet is trained on images with only 3 channels
        if(images.shape[1] == 3):
        
            # Load images and labels to device - again GPU!
            images = images.to(device)
            labels = labels.to(device)
#             labels = labels.reshape((1,labels.shape[0])) # [N, 1] - to match with preds shape
            labels = labels.to(torch.long)

            # Forward
            preds = model(images)
            _, max_pred = torch.max(preds, 1)
            
            # Calculating Loss
            _loss = criterion(preds, labels)
            loss = _loss.item()
            epoch_loss.append(loss)

            # Calculating Accuracy
            acc = get_accuracy(max_pred, labels)
            epoch_acc.append(acc)
    
    # Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    # Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    # Log the results
    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(epoch_acc)
    val_logs["time"].append(total_time)
    
    # Save the best model
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(),"resnet50_best.pth")
        
    return epoch_loss, epoch_acc, total_time, best_val_acc
        

## ResNet50

Let's goooooo - Chinmay 2022

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [23]:
def build_model(model):
    model.aux_logits=False
    for param in model.parameters():
        param.requires_grad = False
    n_classes = 120
    n_inputs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(n_inputs, n_classes),
        nn.LogSoftmax(dim=1))
    model.cuda()
#     model.class_to_idx = all_data.class_to_idx
#     model.idx_to_class = {
#         idx: class_
#         for class_, idx in model.class_to_idx.items()
#     }

In [24]:
n_classes = 120

model = resnet50(pretrained = True)
build_model(model)


# for param in model.parameters():
#     param.requires_grad = False

# num_ftrs = model.fc.in_features

# # model.fc = nn.Linear(num_ftrs, 120)

# model.fc = nn.Sequential(
# nn.Linear(num_ftrs, 1024),
# nn.ReLU(),
# nn.Dropout(0.4),
# nn.Linear(1024, n_classes),
# nn.LogSoftmax(dim=1))

# repeat experiment and change head... 



In [25]:
# Optimizer
learning_rate = 0.001
momentum = 0.9

optimizer=optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) #step_size = 5, gamma = 0.5)

# Loss Function
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()

# Logs
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
model.to(device)

# No of epochs 
epochs = 10

In [26]:
# Imagenet

In [ ]:
best_val_acc = 0 # this will be computed in the validation step

for epoch in range(epochs):
    
    # Training
    loss, acc, _time = train_one_epoch(train_dataloader)
    
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    # Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_dataloader, best_val_acc)
    
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    



Training
Epoch 1
Loss : 4.5534
Acc : 0.1183
Time : 343.6766

Validating
Epoch 1
Loss : 4.2062
Acc : 0.2719
Time : 70.0354

Training
Epoch 2
Loss : 3.9606
Acc : 0.3931
Time : 285.3153

Validating
Epoch 2
Loss : 3.6224
Acc : 0.5279
Time : 45.3476

Training
Epoch 3
Loss : 3.4413
Acc : 0.5737
Time : 349.4309

Validating
Epoch 3
Loss : 3.1266
Acc : 0.6434
Time : 69.0597

Training
Epoch 4
Loss : 2.9954
Acc : 0.6655
Time : 349.1805

Validating
Epoch 4
Loss : 2.7155
Acc : 0.7106
Time : 71.1366

Training
Epoch 5
Loss : 2.6167
Acc : 0.7281
Time : 348.0804

Validating
Epoch 5
Loss : 2.3551
Acc : 0.7625
Time : 68.1187

Training
Epoch 6
Loss : 2.3059
Acc : 0.763
Time : 346.4564

Validating
Epoch 6
Loss : 2.0721
Acc : 0.7809
Time : 67.9894

Training
Epoch 7
Loss : 2.0545
Acc : 0.7903
Time : 346.5604

Validating
Epoch 7
Loss : 1.847
Acc : 0.8069
Time : 68.2421

Training
Epoch 8
Loss : 1.928
Acc : 0.8033
Time : 347.7819

Validating
Epoch 8
Loss : 1.8263
Acc : 0.7987
Time : 71.3052

Training
Epoch 9
L

In [ ]:
# print(transform)
# best_val_acc

In [ ]:
experiment_df = pd.read_csv("../../experiments/"+YOUR_NAME+"/experiments_df.csv")
experiment_df.head()

In [ ]:
# Save experiment settings and result. 
# criterion
experiment_df = pd.read_csv("../../experiments/"+YOUR_NAME+"/experiments_df.csv")
experiment_df.loc[len(experiment_df.index)] = [epochs, best_val_acc, learning_rate, batch_size, momentum, transform_train, model]
#                               




In [ ]:
experiment_df.head(20)

In [ ]:
# experiment_df.head()
# experiment_df.loc[len(experiment_df.index)] = [epochs, best_val_acc, learning_rate, batch_size, momentum, transform_train, model]
experiment_df.head(20)
# [epochs, best_val_acc, learning_rate, batch_size, momentum, transform_train, model]

In [ ]:
experiment_df.to_csv("../../experiments/"+YOUR_NAME+"/experiments_df.csv", index=False) 

In [ ]:
# Save model
torch.save(model.state_dict(),f"../../experiments/{YOUR_NAME}/resnet50_best.pth")